# Lenia

## A continuous cellular automata framework

### Overview

![](../assets/lenia_orbium_summary.png)

Lenia ([Chan 2019](https://www.complex-systems.com/abstracts/v28_i03_a01/))  is a continuous cellular automata (CA) framework with a state-plus-update format that closely follow Euler's method. 

To compute the next state of cells $A_t$ in Lenia involves modifying the current state by the results of an update function $U(A_t)$ weighted by step size $\Delta t$

$$
A_{t+ \Delta t} = A_t + \Delta t * U(A_t)
$$

The update function acts on the neighborhoods, the result of convolving (denoted by $\circledast$) the cell states $A_t$ with a neighborhood kernel $K_n$. 

$$
U(A_t) = G(K_n \circledast A_t)
$$

### Neighborhoods

Neighborhood kernels can have arbitrary values, but most CA in Lenia combine one or more radial Gaussians. 

$$
K_n(r)  = a {e} ^{- \frac{(r - \mu_K)^2}{2 \sigma_K^2} }
$$

Kernels are normalized to sum to 1.0, allowing kernel scaling without changing the magnitude of convolution results. 

$ \frac{1}{a} = \sum_{i,j} e^{-\frac{(\sqrt{i^2+j^2} - b)^2}{c^2}} $

![examples of neighborhood kernels](../assets/neighborhood_kernels.png)

### Updates

For many Lenia CA, the update function is a Gaussian, shifted to output values in the range from -1 to 1. 

$$
G(x) = 2 {e} ^{- \frac{(x - \mu_G)^2}{2 \sigma_G^2} } - 1
$$


<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/lenia_4.gif"> -->

<!-- local image -->
![Orbium cellular automaton gliders](../assets/lenia_4.gif)

In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="Lenia CA animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap

    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(grid[0,0])[:,:,:3]
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    #global ax
    
    grid = ca(grid)
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(grid[0,0])[:,:,:3]
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()

In [ ]:
lib = Librarian(verbose=False)

save_figs = False

num_frames = 384
grid_dim = 67

In [ ]:
pattern_name = "orbium_orbium000"
#'synorbium_orbium000'

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])

grid = ca.initialize_grid(dim=grid_dim)

grid[:,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

p, m = lib.load("synorbium_orbium000")
grid[:,:,-p.shape[-2]-3:-3,:p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid)
plt.show()

if save_figs:
    save_path = os.path.join("..","assets", "lenia_4.gif")
    matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).save(save_path)
    
IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())